<a href="https://colab.research.google.com/github/kairamilanifitria/NLP-Projects/blob/main/Project%202%20Text%20Summarization/Fix/3_Modelling_T5_Extractive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extractive Model

In [1]:
!pip install datasets transformers torch accelerate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel
from transformers import pipeline

In [6]:
# Load the dataset
train_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/fix_dataset/30k/train_df.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/fix_dataset/30k/dev_df.csv')

# Basic overview
print(train_df.shape)
print(train_df.info())
print(train_df.isnull().sum())

print(dev_df.shape)
print(dev_df.info())
print(dev_df.isnull().sum())


(29842, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29842 entries, 0 to 29841
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Unnamed: 0                  29842 non-null  int64 
 1   original_text               29842 non-null  object
 2   abstractive_summary         29842 non-null  object
 3   extractive_summary          29842 non-null  object
 4   original_text_length        29842 non-null  int64 
 5   abstractive_summary_length  29842 non-null  int64 
 6   extractive_summary_length   29842 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 1.6+ MB
None
Unnamed: 0                    0
original_text                 0
abstractive_summary           0
extractive_summary            0
original_text_length          0
abstractive_summary_length    0
extractive_summary_length     0
dtype: int64
(3000, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
D

In [7]:
columns_to_delete = ['Unnamed: 0', 'abstractive_summary', 'original_text_length', 'abstractive_summary_length', 'extractive_summary_length']

# Delete the specified columns from the DataFrame
train_df = train_df.drop(columns=columns_to_delete)
dev_df = dev_df.drop(columns=columns_to_delete)

In [8]:
train_df.iloc[1]

,1
original_text,liputan6 com jakarta karyawan pt angkasa pura ...
extractive_summary,liputan6 com jakarta karyawan pt angkasa pura ...


In [9]:
dev_df.iloc[1]

,1
original_text,liputan6 com jakarta peruntungan ahmad taufik ...
extractive_summary,seperti tak mau ketinggalan presiden abdurrahm...


In [10]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load pre-trained T5 model for text summarization (extractive)
model_name = "cahya/t5-base-indonesian-summarization-cased"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/793k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [12]:
# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

In [13]:
from transformers import TrainingArguments, Trainer

# Convert pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

def preprocess_function_extractive(examples):

    extractive_summary = [
        " ".join(summary) if isinstance(summary, list) else summary
        for summary in examples["extractive_summary"]
    ]  # List comprehension for handling list summaries

    # Tokenize the text input
    model_inputs = tokenizer(
        examples["original_text"], max_length=512, truncation=True, padding="max_length"
    )

    # Tokenize the combined summary (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            extractive_summary, max_length=256, truncation=True, padding="max_length"
        )

    # Add the tokenized labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
# Apply preprocessing to the train and dev datasets
tokenized_train_datasets = train_dataset.map(preprocess_function_extractive, batched=True)
tokenized_dev_datasets = dev_dataset.map(preprocess_function_extractive, batched=True)

Map:   0%|          | 0/29842 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [15]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    push_to_hub=False,
    fp16=True,
    save_steps=500,           # Save the model every 500 steps
    eval_steps=500,           # Evaluate every 500 steps
    logging_dir="./logs",     # Directory for logs
    logging_steps=100,        # Log every 100 steps
)


# Define data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_datasets,  # Use tokenized train dataset
    eval_dataset=tokenized_dev_datasets,     # Use tokenized dev dataset for validation
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [16]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.048700,0.037730
2,0.042400,0.034665
3,0.038600,0.034300


TrainOutput(global_step=11193, training_loss=0.05814688503715592, metrics={'train_runtime': 9495.1697, 'train_samples_per_second': 9.429, 'train_steps_per_second': 1.179, 'total_flos': 5.451756411027456e+16, 'train_loss': 0.05814688503715592, 'epoch': 3.0})

In [17]:
import os

# Define the path in Google Drive where you want to save the model
output_dir = '/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/T5:Extractive'

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the trained model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Model saved to {output_dir}")

Model saved to /content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/T5:Extractive


In [18]:
test_df = pd.read_csv('/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/fix_dataset/30k/test_df.csv')

In [19]:
columns_to_delete = ['Unnamed: 0', 'abstractive_summary', 'original_text_length', 'abstractive_summary_length', 'extractive_summary_length']
# Delete the specified columns from the DataFrame
test_df = test_df.drop(columns=columns_to_delete)
test_df.head()

,original_text,extractive_summary
0,liputan6 com jakarta ratusan mahasiswa se jabo...,liputan6 com jakarta ratusan mahasiswa se jabo...
1,liputan6 com palembang puluhan orang perwakila...,mereka menuntut perusahaan mengembalikan tanah...
2,liputan6 com jakarta nangroe aceh darussalam d...,liputan6 com jakarta nangroe aceh darussalam d...
3,liputan6 com jakarta tidak ada perbedaan peraw...,perbedaan hanya pada pelayanan kepada para pen...
4,liputan6 com jakarta menteri luar negeri demis...,pernyataan itu disampaikan alwi seusai bertemu...


In [20]:
test_df.shape

(2991, 2)

In [21]:
!pip install tqdm  # Install tqdm if you haven't already
from tqdm import tqdm


def generate_extractive_summaries_for_df(df):
    model_path = "/content/drive/MyDrive/Bootcamp AI/Dataset/Task2 : Text Summarization/T5:Extractive"
    summarizer = T5ForConditionalGeneration.from_pretrained(model_path).to(device)  # Load your fine-tuned model
    tokenizer = T5Tokenizer.from_pretrained(model_path)

    # Lists to store both types of summaries
    extractive_summaries = []

    # Loop through the test dataset and generate both summaries
    for text in tqdm(df['original_text'], desc="Generating Summaries"):
        try:
            # Tokenize the input text
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

            # Generate the summary from the model
            summary_ids = summarizer.generate(**inputs, max_length=200)  # Adjust max_length as needed
            extractive_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            # Append the generated summaries to their respective lists
            extractive_summaries.append(extractive_summary.strip())

        except Exception as e:
            print(f"Error generating summary for text: {text[:50]}...")  # Print a message for debugging
            print(f"Error: {e}")  # Print the error
            # Append empty strings to maintain consistent output length
            extractive_summaries.append("")

    # Return both summaries in a DataFrame
    return pd.DataFrame({
        'extractive_summary': extractive_summaries
    })

# Assuming test_df_subset is already defined
extractive_summary_T5_df = generate_extractive_summaries_for_df(test_df)

extractive_summary_T5_df

Generating Summaries: 100%|██████████| 2991/2991 [51:06<00:00,  1.03s/it]


,extractive_summary
0,liputan6 com jakarta ratusan mahasiswa se jabo...
1,mereka menuntut perusahaan mengembalikan tanah...
2,kepala staf tni angkatan darat jenderal tni en...
3,liputan6 com jakarta tidak ada perbedaan peraw...
4,liputan6 com jakarta menteri luar negeri demis...
...,...
2986,liputan6 com jakarta tim otonomi khusus papua ...
2987,liputan6 com jakarta buku kumpulan lukisan kar...
2988,liputan6 com jakarta banyak cara dalam mensosi...
2989,liputan6 com jakarta kepolisian daerah metro j...


In [23]:
from google.colab import files

csv_file_path = 'extractive_summary_T5_df.csv'
extractive_summary_T5_df.to_csv(csv_file_path, index=False)

files.download(csv_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **ROUGE CALCULATION**

In [24]:
!pip install datasets transformers torch accelerate rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ce928a0b44ea64713cc087ae614c2cbe09e020fb73b31f978f48c7be21381e8b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [25]:
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorForSeq2Seq, EncoderDecoderModel, AutoModelForSeq2SeqLM, AutoTokenizer # Import AutoModelForSeq2SeqLM and AutoTokenizer
from transformers import pipeline

In [26]:
# Check if GPU is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [32]:
# calculate ROUGE-1, ROUGE-2 and ROUGE-L score in test_df column "extractive_summary" as references, and extractive_summary_BERT_df as candidate

from rouge_score import rouge_scorer

def calculate_rouge_scores(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = []
    for ref, cand in zip(references, candidates):
        scores = scorer.score(ref, cand)
        rouge_scores.append(scores)
    return rouge_scores

references = test_df['extractive_summary'].tolist()
candidates = extractive_summary_T5_df['extractive_summary'].tolist()

rouge_scores = calculate_rouge_scores(references, candidates)

# Calculate average scores
rouge1_scores = [score['rouge1'].fmeasure for score in rouge_scores]
rouge2_scores = [score['rouge2'].fmeasure for score in rouge_scores]
rougeL_scores = [score['rougeL'].fmeasure for score in rouge_scores]

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores) if rouge1_scores else 0
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores) if rouge2_scores else 0
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores) if rougeL_scores else 0

print(f"Average ROUGE-1 fine-tuned T5 model extractive summary")
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

Average ROUGE-1 fine-tuned T5 model extractive summary
Average ROUGE-1: 0.5251607346805842
Average ROUGE-2: 0.4489396826470044
Average ROUGE-L: 0.4917028242319308


comparison to raw model

In [28]:
!pip install tqdm  # Install tqdm if you haven't already
from tqdm import tqdm


def generate_extractive_summaries_for_df(df):
    model_path = "cahya/t5-base-indonesian-summarization-cased"
    summarizer = T5ForConditionalGeneration.from_pretrained(model_path).to(device)  # Load your fine-tuned model
    tokenizer = T5Tokenizer.from_pretrained(model_path)

    # Lists to store both types of summaries
    extractive_summaries = []

    # Loop through the test dataset and generate both summaries
    for text in tqdm(df['original_text'], desc="Generating Summaries"):
        try:
            # Tokenize the input text
            inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)

            # Generate the summary from the model
            summary_ids = summarizer.generate(**inputs, max_length=200)  # Adjust max_length as needed
            extractive_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

            # Append the generated summaries to their respective lists
            extractive_summaries.append(extractive_summary.strip())

        except Exception as e:
            print(f"Error generating summary for text: {text[:50]}...")  # Print a message for debugging
            print(f"Error: {e}")  # Print the error
            # Append empty strings to maintain consistent output length
            extractive_summaries.append("")

    # Return both summaries in a DataFrame
    return pd.DataFrame({
        'extractive_summary': extractive_summaries
    })

# Assuming test_df_subset is already defined
extractive_summary_T5_raw = generate_extractive_summaries_for_df(test_df)

extractive_summary_T5_raw

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Generating Summaries: 100%|██████████| 2991/2991 [2:18:45<00:00,  2.78s/it]


,extractive_summary
0,Ratusan mahasiswa se jabotabek memperingati ti...
1,Puluhan warga desa kota baru dan desa pagar gu...
2,Kepala Staf tni angkatan darat jenderal tni en...
3,jakarta tidak ada perbedaan perawatan antara k...
4,Menteri Luar Negeri demisioner alwi shihab men...
...,...
2986,Tim otonomi khusus papua mendesak dpr dan peme...
2987,Liputan6 com jakarta buku kumpulan lukisan kar...
2988,Penyaluran cerita percintaan antara sami un da...
2989,kepolisian daerah metro jaya rabu 28 11 mulai ...


In [29]:
from google.colab import files

csv_file_path = 'extractive_summary_T5_raw.csv'
extractive_summary_T5_df.to_csv(csv_file_path, index=False)

files.download(csv_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
# calculate ROUGE-1, ROUGE-2 and ROUGE-L score in test_df column "extractive_summary" as references, and extractive_summary_BERT_df as candidate

from rouge_score import rouge_scorer

def calculate_rouge_scores(references, candidates):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = []
    for ref, cand in zip(references, candidates):
        scores = scorer.score(ref, cand)
        rouge_scores.append(scores)
    return rouge_scores

references = test_df['extractive_summary'].tolist()
candidates = extractive_summary_T5_raw['extractive_summary'].tolist()

rouge_scores = calculate_rouge_scores(references, candidates)

# Calculate average scores
rouge1_scores = [score['rouge1'].fmeasure for score in rouge_scores]
rouge2_scores = [score['rouge2'].fmeasure for score in rouge_scores]
rougeL_scores = [score['rougeL'].fmeasure for score in rouge_scores]

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores) if rouge1_scores else 0
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores) if rouge2_scores else 0
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores) if rougeL_scores else 0

print(f"Average ROUGE-1 raw T5 model extractive summary")
print(f"Average ROUGE-1: {avg_rouge1}")
print(f"Average ROUGE-2: {avg_rouge2}")
print(f"Average ROUGE-L: {avg_rougeL}")

Average ROUGE-1 raw T5 model extractive summary
Average ROUGE-1: 0.41242164508185103
Average ROUGE-2: 0.3673287351930601
Average ROUGE-L: 0.3901658464137844
